In [2]:
pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/34/7c/8a5043f362b0a55f07812a0db3f86092cdbd0fe41b933d7bc6fce3ab6c15/opencv_python-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 13.3 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import cv2

In [3]:
def resize(img, perc=0.5):
    """Utility function to resize the image"""
    width = int(img.shape[1] * perc)
    height = int(img.shape[0] * perc)
    resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
    return resized


def dark_channel(img, sz=15):
    """Calculates the dark channel in step 1"""
    b, g, r = cv2.split(img)
    dc = r - cv2.max(b, g)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark


def airlight(img, dc):
    """Calculates the air-light in step 4"""
    [h, w] = img.shape[:2]
    imgsz = h * w
    imgvec = img.reshape(imgsz, 3)
    dcvec = dc.reshape(imgsz)

    indices = dcvec.argsort()
    return imgvec[indices[0]]


def transmission_estimate(dc):
    """Calculates the transmission map as mentioned in step 2"""
    te = dc + (1 - np.max(dc))
    return te


def guided_filter(img, p, r, eps):
    """Helper function to refine the transmission"""
    mean_I = cv2.boxFilter(img, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(img * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(img * img, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))

    q = mean_a * img + mean_b
    return q


def transmission_refine(img, et):
    """Refines the transmission map as mentioned in step 3"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = guided_filter(gray, et, r, eps)
    return t


def recover(img, t, A):
    """Finally recovers the image using the image, the transmission map and air-light"""
    res = np.empty(img.shape, img.dtype)
    for index in range(0, 3):
        res[:, :, index] = (img[:, :, index] - A[index]) / t + A[index]

    return res


def normalize_image(img):
    """Utility function to normalize the intensities of the pixels in the image"""
    img = img - img.min()
    img = img / img.max() * 255
    img = np.uint8(img)
    return img


def dummy(x):
    pass


In [3]:
if __name__ == '__main__':
    filename = 'color_img1663755872636549118.jpg'
    scale = 1

    src = cv2.imread(f'/home/coena98/olter/images/{filename}')
    src = resize(src, scale)

    img = src.astype('float64') / 255

#     cv2.namedWindow('controls')
#     cv2.createTrackbar('ksize', 'controls', 1, 600, dummy)

#     while True:
# #         ksize = int(cv2.getTrackbarPos('ksize', 'controls')) + 1
#         ksize = int(128)

#         dc = dark_channel(img, ksize)
#         te = transmission_estimate(dc)
#         tr = transmission_refine(src, te)
#         A = airlight(img, tr)
#         result = recover(img, tr, A)
#         result = normalize_image(result)

#         cv2.imshow('Transmission estimate', te)
#         cv2.imshow('Transmission refined', tr)
#         cv2.imshow('Input', img)
#         cv2.imshow('Result', result)

#         k = cv2.waitKey(1) & 0xFF
#         if k == 27:
#             break
    ksize = int(128)

    dc = dark_channel(img, ksize)
    te = transmission_estimate(dc)
    tr = transmission_refine(src, te)
    A = airlight(img, tr)
    result = recover(img, tr, A)
    result = normalize_image(result)

    cv2.imwrite(f'/home/coena98/olter/enhanced_images/{filename}', result)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread (0x2b07bc0).
Cannot move to target thread (0x29414f0)

QObject::moveToThread: Current thread (0x29414f0) is not the object's thread

In [25]:
from os import listdir
from os.path import isfile, join
mypath = '/home/coena98/olter/images/'
onlyfiles = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyimgs = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [27]:
onlyimgs[0]

'color_img1663755873856968774.jpg'

In [35]:

scale = 1
for i in range(len(onlyfiles)):
    
    src = cv2.imread(onlyfiles[i])
    src = resize(src, scale)

    img = src.astype('float64') / 255

    ksize = int(128)

    dc = dark_channel(img, ksize)
    te = transmission_estimate(dc)
    tr = transmission_refine(src, te)
    A = airlight(img, tr)
    result = recover(img, tr, A)
    result = normalize_image(result)
    filename = onlyimgs[i]


    cv2.imwrite(f'/home/coena98/olter/enhanced_images/{filename}', result)

In [ ]:
onlyfiles

In [1]:
pip install image_dehazer

  Preparing metadata (setup.py) ... done
  Created wheel for image_dehazer: filename=image_dehazer-0.0.9-py3-none-any.whl size=6394 sha256=148e17ca9ad70870199250c66caf057a4cd7ee777e0d537b49773bfcb3e26acc
  Stored in directory: /home/coena98/.cache/pip/wheels/a4/8d/c4/26fd3a025648d889f5194d3a5e14520096f67ef44f5b21cc3b
Successfully built image_dehazer
Note: you may need to restart the kernel to use updated packages.


In [1]:
import image_dehazer # Load the library
import cv2

In [4]:
HazeImg = cv2.imread('/home/coena98/olter/images/color_img1663755872636549118.jpg')# read input image -- (**must be a color image**)
HazeCorrectedImg = image_dehazer.remove_haze(HazeImg, showHazeTransmissionMap=False)# Remove Haze


In [5]:
cv2.imwrite(f'/home/coena98/olter/enhanced_images/color_img1663755872636549118_1.jpg', HazeCorrectedImg[0])
# cv2.imshow('input image', HazeImg);# display the original hazy image
# cv2.imshow('enhanced_image', HazeCorrectedImg[1]);# display the result
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [6]:
cv2.imwrite(f'/home/coena98/olter/enhanced_images/color_img1663755872636549118_2.jpg', HazeCorrectedImg[1])

True

In [5]:
cv2.imshow('input image', HazeImg);# display the original hazy image
cv2.imshow('enhanced_image', HazeCorrectedImg);# display the result
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(f'/home/coena98/olter/enhanced_images/color_img1663755888903749289.jpg', HazeCorrectedImg)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread (0x2916d10).
Cannot move to target thread (0x2841510)

QObject::moveToThread: Current thread (0x2841510) is not the object's thread

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numerical tuple
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'
